In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity_recognition_exp.zip' -d '/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /'

Archive:  /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity_recognition_exp.zip
   creating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/
  inflating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/.DS_Store  
   creating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /__MACOSX/
   creating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /__MACOSX/Activity recognition exp/
  inflating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /__MACOSX/Activity recognition exp/._.DS_Store  
  inflating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_accelerometer.csv  
  inflating: /content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_gyroscope.csv  
  inflating: /content/drive/My Drive/C

In [0]:
data1 = pd.read_csv('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_accelerometer.csv')		#Loading Accelerometer data
data2 = pd.read_csv('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /Activity recognition exp/Phones_accelerometer.csv')			#Loading Gyroscope data

In [0]:
length1 = len(data1)
length2 = len(data2)
length = min(length1, length2)						#To make the length of the merged data equal to minimum of the two data 


In [0]:
data1 = data1.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User'], axis=1)		#Dropping the unnecessary fields
data2 = data2.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User','Model','Device'], axis=1)


In [0]:
data2.columns = ['x1', 'y1', 'z1', 'gt1']

In [0]:
data = pd.concat([data1, data2], axis=1)

In [0]:
data.head

<bound method NDFrame.head of                  x         y         z  ...        y1        z1    gt1
0        -5.958191  0.688065  8.135345  ...  0.688065  8.135345  stand
1        -5.952240  0.670212  8.136536  ...  0.670212  8.136536  stand
2        -5.995087  0.653549  8.204376  ...  0.653549  8.204376  stand
3        -5.942718  0.676163  8.128204  ...  0.676163  8.128204  stand
4        -5.991516  0.641647  8.135345  ...  0.641647  8.135345  stand
...            ...       ...       ...  ...       ...       ...    ...
13062470  1.379043  0.153227  9.959755  ...  0.153227  9.959755   bike
13062471  1.379043  0.153227  9.806528  ...  0.153227  9.806528   bike
13062472  1.532270  0.153227  9.806528  ...  0.153227  9.806528   bike
13062473  1.532270  0.000000  9.959755  ...  0.000000  9.959755   bike
13062474  1.379043  0.000000  9.959755  ...  0.000000  9.959755   bike

[13062475 rows x 10 columns]>

In [0]:
to_drop = ['null']									#To drop the null values fro both data1 and data2
data = data[~data['gt'].isin(to_drop)]
data = data[~data['gt1'].isin(to_drop)]

In [0]:
data = data.drop(labels = ['gt1'], axis=1)

In [0]:
cols_to_norm = ['x','y','z', 'x1','y1','z1']		#Normalizing the columns
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [0]:
y = data[['gt']]									#Extracting only the action values
y= pd.get_dummies(y)
# data = data.drop(labels = ['gt'], axis = 1)

In [0]:
sample_rate = 50  # number of observation per second based on dataset documentation

sliding_size = int(.2 * sample_rate)  # number of skipped datapoints to start next window
win_size=128

In [0]:
data.columns

Index(['x', 'y', 'z', 'Model', 'Device', 'gt', 'x1', 'y1', 'z1'], dtype='object')

In [0]:
def sliding_window(overlap=False):
 
  if overlap:
        step_size = sliding_size  # for Overlapping technique
  else:

    step_size = win_size  # for Non-overlapping technique
  y_list=[]
  x_list=[]      
  for i in range(0, data.shape[0], step_size):
    start = i
    end = start + step_size
    if end<data.shape[0]:
      
      label = data.loc[i]['gt']
      X_point = np.zeros((step_size,6))
      # acc_x = data[i: i + N_TIME_STEPS, 'x']
      # acc_y = data[i: i + N_TIME_STEPS, 'y']
      # acc_z = data[i: i + N_TIME_STEPS, 'z']
      # gyr_x = data[i: i + N_TIME_STEPS, 'x1']
      # gyr_y = data[i: i + N_TIME_STEPS, 'y1']
      # gyr_z = data[i: i + N_TIME_STEPS, 'z1']
      # print(data.iloc[i: i + step_size, 0])
      X_point[:,0] = data.iloc[i: i + step_size, 0]
      X_point[:,1] = data.iloc[i: i + step_size, 1]
      X_point[:,2] = data.iloc[i: i + step_size, 2]
      X_point[:,3] = data.iloc[i: i + step_size, 6]
      X_point[:,4] = data.iloc[i: i + step_size, 7]
      X_point[:,5] = data.iloc[i: i + step_size, 8]
      x_list.append(X_point)
      y_list.append(label)
    # print(acc_x)
    # break
    # while until <= segment_end:
    #   X_point = np.zeros((win_size,6))
    #   # print(acc_x[segment_start:until])
    #   X_point[:,0] = acc_x[segment_start:until]
    #   X_point[:,1] = acc_y[segment_start:until]
    #   X_point[:,2] = acc_z[segment_start:until]
    #   X_point[:,3] = gyr_x[segment_start:until]
    #   X_point[:,4] = gyr_y[segment_start:until]
    #   X_point[:,5] = gyr_z[segment_start:until]
    #   x_list.append(X_point)
    #   print('xpoint',X_point)
    #   print('xlist',x_list)


    #   segment_start+=step_size
    #   until+=step_size
    #   y_list.append(label) 
  return y_list,  x_list 


         

In [0]:
overlap=True
step_size=None
if overlap:
        step_size = sliding_size  # for Overlapping technique
else:
  step_size = win_size  # for Non-overlapping technique

In [0]:
Y,X=sliding_window(overlap)

In [0]:
X_train = np.zeros((len(Y),step_size,6))
X_train[:,:]=[x for x in X]

In [0]:
X_train.shape

(1306247, 10, 6)

In [0]:
type(Y)

list

In [0]:
y_df=df = pd.DataFrame(Y)

In [0]:
y_dummy= pd.get_dummies(y_df)

In [0]:
y_np = np.array(y_dummy)

In [0]:
y_np

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=uint8)

In [0]:
Y_train=np.zeros(len(Y))
for y in y_np:
   index=np.where(y == 1)
   print(index)
   Y_train[:]=index



(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array([4]),)
(array

ValueError: ignored

In [0]:
Y_train

array([4., 4., 4., ..., 4., 4., 4.])

In [0]:
np.savez_compressed('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /UCI_HHAR_X.npz', X_train)

In [0]:
np.savez_compressed('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /UCI_HHAR_Y.npz', y_np)